In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
library(DESeq2)
library(phyloseq)
library(plyr); library(dplyr)
library(ggplot2)
library(doParallel)
library(foreach)

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following object is masked from ‘package:stats’:

    xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, as.vector, cbind, colnames,
    do.call, duplicated, eval, evalq, Filter, Find, get, intersect,
    is.unsorted, lapply, Map, mapply, match, mget, order, paste, pmax,
    pmax.int, pmin, pmin.int, Position, rank, rbind, Reduce, rep.int,
    rownames, sapply, setdiff, sort, table, tapply, union, unique,
    unlist, unsplit

Loading required package: IRanges
Loading required package: GenomicRanges
Loading required package:

In [4]:
%%R

asNumeric = function(x) { as.numeric(as.character(x)) }

physeq = import_biom("/var/seq_data/priming_exp/data/otu_table_wtax.biom", "/var/seq_data/priming_exp/data/otusn.tree")
sample.data = import_qiime_sample_data("/var/seq_data/priming_exp/data/allsample_metadata_nomock.txt")
sample.data$Density = asNumeric(sample.data$Density)
physeq = merge_phyloseq(physeq,sample.data)
physeq

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 10361 taxa and 466 samples ]
sample_data() Sample Data:       [ 466 samples by 11 sample variables ]
tax_table()   Taxonomy Table:    [ 10361 taxa by 8 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 10361 tips and 10360 internal nodes ]


In [8]:
%%R
density.min = 1.7125
density.max = 1.755

md = sample_data(physeq)
#physeq.heavy = prune_samples((md$Density >= density.min)&
#                             (md$Density <= density.max), physeq)

get_l2fc = function(physeq.obj, Treatment, Day) {
    physeq.md = sample_data(physeq.obj)
    p = prune_samples((physeq.md$Bulk == 0)& (physeq.md$Treatment %in% c("12CC",as.character(Treatment)))&
                      (physeq.md$Day == Day)&
                      (physeq.md$Density >= density.min)&
                      (physeq.md$Density <= density.max), physeq.obj)
    dds = phyloseq_to_deseq2(p, ~Treatment)
    dds = DESeq(dds, quiet = TRUE, fitType = "local")
    r = results(dds, independentFiltering = FALSE)
    r$OTU = rownames(r)
    d = data.frame(r[, c("OTU","log2FoldChange", "lfcSE")])
    TT = data.frame(tax_table(p))
    TT$OTU = rownames(TT)
    d = left_join(d, TT)
    d
}

m = expand.grid(Day = c(14, 28, 45),
                Treatment = c("13CC"))

#m = m[!((m$Day == 1)&(m$Treatment == "13CCPS")),]

registerDoParallel(10)

df.l2fc.CC = tbl_df(mdply(m, 
                       get_l2fc, 
                       .parallel = TRUE,
                       physeq.obj = physeq.heavy))

df.l2fc.CC

Source: local data frame [31,083 x 13]

   Day Treatment       OTU log2FoldChange    lfcSE    Rank1          Rank2
1   14      13CC OTU.14353             NA       NA Bacteria  Acidobacteria
2   14      13CC OTU.11642     -0.9017218 1.536176 Bacteria             NA
3   14      13CC  OTU.9532             NA       NA Bacteria Proteobacteria
4   14      13CC OTU.12098             NA       NA Bacteria Proteobacteria
5   14      13CC OTU.12051             NA       NA Bacteria Proteobacteria
6   14      13CC OTU.12843             NA       NA Bacteria Proteobacteria
7   14      13CC OTU.12643     -0.3962491 1.532219 Bacteria  Acidobacteria
8   14      13CC  OTU.9465             NA       NA Bacteria  Acidobacteria
9   14      13CC OTU.12562      0.7793390 1.530693 Bacteria  Acidobacteria
10  14      13CC OTU.11557             NA       NA Bacteria  Acidobacteria
.. ...       ...       ...            ...      ...      ...            ...
Variables not shown: Rank3 (fctr), Rank4 (fctr), Rank5 (fctr

In [9]:
%%R
density.min = 1.7125
density.max = 1.755

md = sample_data(physeq)
#physeq.heavy = prune_samples((md$Density >= density.min)&
#                             (md$Density <= density.max), physeq)

get_l2fc = function(physeq.obj, Treatment, Day) {
    physeq.md = sample_data(physeq.obj)
    p = prune_samples((physeq.md$Bulk == 0)& (physeq.md$Treatment %in% c("12C100",as.character(Treatment)))&
                      (physeq.md$Day == Day)&
                      (physeq.md$Density >= density.min)&
                      (physeq.md$Density <= density.max), physeq.obj)
    dds = phyloseq_to_deseq2(p, ~Treatment)
    dds = DESeq(dds, quiet = TRUE, fitType = "local")
    r = results(dds, independentFiltering = FALSE)
    r$OTU = rownames(r)
    d = data.frame(r[, c("OTU","log2FoldChange", "lfcSE")])
    TT = data.frame(tax_table(p))
    TT$OTU = rownames(TT)
    d = left_join(d, TT)
    d
}

m.100 = expand.grid(Day = c(14, 28, 45),
                Treatment = c("13C100"))

#m = m[!((m$Day == 1)&(m$Treatment == "13CCPS")),]

registerDoParallel(10)

df.l2fc.100 = tbl_df(mdply(m.100, 
                       get_l2fc, 
                       .parallel = TRUE,
                       physeq.obj = physeq.heavy))

df.l2fc.100

Source: local data frame [31,083 x 13]

   Day Treatment       OTU log2FoldChange    lfcSE    Rank1          Rank2
1   14    13C100 OTU.14353             NA       NA Bacteria  Acidobacteria
2   14    13C100 OTU.11642       1.866406 1.398122 Bacteria             NA
3   14    13C100  OTU.9532             NA       NA Bacteria Proteobacteria
4   14    13C100 OTU.12098             NA       NA Bacteria Proteobacteria
5   14    13C100 OTU.12051             NA       NA Bacteria Proteobacteria
6   14    13C100 OTU.12843             NA       NA Bacteria Proteobacteria
7   14    13C100 OTU.12643             NA       NA Bacteria  Acidobacteria
8   14    13C100  OTU.9465             NA       NA Bacteria  Acidobacteria
9   14    13C100 OTU.12562             NA       NA Bacteria  Acidobacteria
10  14    13C100 OTU.11557             NA       NA Bacteria  Acidobacteria
.. ...       ...       ...            ...      ...      ...            ...
Variables not shown: Rank3 (fctr), Rank4 (fctr), Rank5 (fctr

In [10]:
%%R
density.min = 1.7125
density.max = 1.755

md = sample_data(physeq)
#physeq.heavy = prune_samples((md$Density >= density.min)&
#                             (md$Density <= density.max), physeq)

get_l2fc = function(physeq.obj, Treatment, Day) {
    physeq.md = sample_data(physeq.obj)
    p = prune_samples((physeq.md$Bulk == 0)& (physeq.md$Treatment %in% c("12C700",as.character(Treatment)))&
                      (physeq.md$Day == Day)&
                      (physeq.md$Density >= density.min)&
                      (physeq.md$Density <= density.max), physeq.obj)
    dds = phyloseq_to_deseq2(p, ~Treatment)
    dds = DESeq(dds, quiet = TRUE, fitType = "local")
    r = results(dds, independentFiltering = FALSE)
    r$OTU = rownames(r)
    d = data.frame(r[, c("OTU","log2FoldChange", "lfcSE")])
    TT = data.frame(tax_table(p))
    TT$OTU = rownames(TT)
    d = left_join(d, TT)
    d
}

m.700 = expand.grid(Day = c(14, 28, 45),
                Treatment = c("13C700"))

#m = m[!((m$Day == 1)&(m$Treatment == "13CCPS")),]

registerDoParallel(10)

df.l2fc.700 = tbl_df(mdply(m.700, 
                       get_l2fc, 
                       .parallel = TRUE,
                       physeq.obj = physeq.heavy))

df.l2fc.700

Source: local data frame [31,083 x 13]

   Day Treatment       OTU log2FoldChange lfcSE    Rank1          Rank2
1   14    13C700 OTU.14353             NA    NA Bacteria  Acidobacteria
2   14    13C700 OTU.11642             NA    NA Bacteria             NA
3   14    13C700  OTU.9532             NA    NA Bacteria Proteobacteria
4   14    13C700 OTU.12098             NA    NA Bacteria Proteobacteria
5   14    13C700 OTU.12051             NA    NA Bacteria Proteobacteria
6   14    13C700 OTU.12843             NA    NA Bacteria Proteobacteria
7   14    13C700 OTU.12643             NA    NA Bacteria  Acidobacteria
8   14    13C700  OTU.9465             NA    NA Bacteria  Acidobacteria
9   14    13C700 OTU.12562             NA    NA Bacteria  Acidobacteria
10  14    13C700 OTU.11557             NA    NA Bacteria  Acidobacteria
.. ...       ...       ...            ...   ...      ...            ...
Variables not shown: Rank3 (fctr), Rank4 (fctr), Rank5 (fctr), Rank6 (fctr),
  Rank7 (fctr), Ran

In [15]:
%%R
df.l2fc.all = rbind(df.l2fc.CC, df.l2fc.100, df.l2fc.700)
head(df.l2fc.all)

Source: local data frame [6 x 13]

  Day Treatment       OTU log2FoldChange    lfcSE    Rank1          Rank2
1  14      13CC OTU.14353             NA       NA Bacteria  Acidobacteria
2  14      13CC OTU.11642     -0.9017218 1.536176 Bacteria             NA
3  14      13CC  OTU.9532             NA       NA Bacteria Proteobacteria
4  14      13CC OTU.12098             NA       NA Bacteria Proteobacteria
5  14      13CC OTU.12051             NA       NA Bacteria Proteobacteria
6  14      13CC OTU.12843             NA       NA Bacteria Proteobacteria
Variables not shown: Rank3 (fctr), Rank4 (fctr), Rank5 (fctr), Rank6 (fctr),
  Rank7 (fctr), Rank8 (fctr)


In [16]:
%%R
write.table(df.l2fc.all, 
            file = "data/l2fc_table_no_spar.csv",
            row.names = FALSE,
            sep = ",")